<a href="https://colab.research.google.com/github/Desire32/ml/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
pr_qst = pd.DataFrame([ ## dataset
    {"question": "What is the historical significance of the mosaic in the apse of the Church of Panagia Aggeloktisti and in what century was it created?",
     "model_response": "",
     },
    {"question": "What architectural features distinguish the Church of Panagia Aggeloktisti and what periods of construction does it unite?",
     "model_response": "",
     },
    {"question": "What does the name Aggeloktisti mean and where does it come from?",
     "model_response": "",
     },
    {"question": "What chapels were added to the church in the Middle Ages and what was their purpose?",
     "model_response": "",
     },
    {"question": "How does the Church of Panagia Aggeloktisti demonstrate the cultural and artistic links between different regions of the Mediterranean?",
     "model_response": "",
     },
    {"question": "What arguments are put forward for the inclusion of the Church of Panagia Aggeloktisti in the UNESCO World Heritage List under criterion (i)?",
     "model_response": "",
     },
    {"question": "What conservation measures have been taken since 1952 to preserve the mosaics and other elements of the church?",
     "model_response": "",
     },
    {"question": "How does the church reflect the religious and spiritual life of the local community over the centuries?",
     "model_response": "",
    },
    {"question": "How does the church of Panagia Aggeloktisti maintain its authenticity and integrity despite the passing centuries and the restorations carried out?",
     "model_response": "",
    }
])

In [ ]:
# llama = "openlm-research/open_llama_3b"
# llama_model = AutoModelForCausalLM.from_pretrained(
#     llama,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     )
# llama_tokenizer = AutoTokenizer.from_pretrained(llama)

# qwen
# !pip install transformers_stream_generator
# !pip install hf_xet # http stuff
# !pip install tiktoken
# qwen = "Qwen/Qwen-1_8B-Chat"
# qwen_model = AutoModelForCausalLM.from_pretrained(
#     qwen,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     trust_remote_code=True ## fix api issues
#     )
# qwen_tokenizer = AutoTokenizer.from_pretrained(qwen)


# # deepseek
# deepseek = "deepseek-ai/deepseek-coder-1.3b-base"
# deepseek_model = AutoModelForCausalLM.from_pretrained(
#     deepseek,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     )
# deepseek_tokenizer = AutoTokenizer.from_pretrained(deepseek)


# falcon
falcon = "tiiuae/falcon-rw-1b"
falcon_model = AutoModelForCausalLM.from_pretrained(
    falcon,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    offload_folder="offload" ## memory issue
    )
falcon_tokenizer = AutoTokenizer.from_pretrained(falcon)


In [ ]:
def gen(question, model, tokenizer):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=500,
        num_return_sequences=1,
        do_sample=False, ## variety, turn off for now
        top_p=0.95,
        temperature=0.7, ## temp
        pad_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("Answer:")[-1].strip()
    return answer


# outputs
llama_responses = [] # 1 minute 17 seconds runtime (3b model)
falcon_responses = [] # 45 seconds (1b model)
qwen_responses = []
deepseek_responses = []

# dataframes
llama_dataframe = pd.DataFrame()
falcon_dataframe = pd.DataFrame()
qwen_dataframe = pd.DataFrame()
deepseek_dataframe = pd.DataFrame()

def output(model, tokenizer, responses, dataframe):
    for question in pr_qst["question"]:
        resp = gen(question, model, tokenizer)
        responses.append(resp)
    dataframe = pr_qst.copy()
    dataframe["model_response"] = responses
    return dataframe

## llama
# output(llama_model, llama_tokenizer, llama_responses, llama_dataframe)

## falcon
output(falcon_model, falcon_tokenizer, falcon_responses, falcon_dataframe)

## qwen
# output(qwen, qwen_tokenizer, qwen_responses, qwen_dataframe)

## deepseek
# output(deepseek, deepseek_tokenizer, deepseek_responses, deepseek_dataframe)